In [36]:
import sys
import warnings
warnings.filterwarnings("ignore")

import os
import codecs
import json
import cv2
from keras.models import load_model
import numpy as np
from keras.preprocessing import image
from tqdm import tqdm 
from utils.datasets import get_labels
from utils.inference import detect_faces
from utils.inference import draw_text
from utils.inference import draw_bounding_box
from utils.inference import apply_offsets
from utils.inference import load_detection_model
from utils.inference import load_image
from utils.preprocessor import preprocess_input
import tensorflow as tf

In [2]:
# hyper-parameters for bounding boxes shape
gender_offsets = (30, 60)
gender_offsets = (10, 10)
emotion_offsets = (20, 40)
emotion_offsets = (0, 0)

In [3]:
detection_model_path = '../trained_models/detection_models/haarcascade_frontalface_default.xml'
emotion_model_path = '../trained_models/emotion_models/fer2013_mini_XCEPTION.102-0.66.hdf5'
gender_model_path = '../trained_models/gender_models/simple_CNN.81-0.96.hdf5'

In [4]:
emotion_labels = get_labels('fer2013')
gender_labels = get_labels('imdb')

In [5]:
face_detection = load_detection_model(detection_model_path)
emotion_classifier = load_model(emotion_model_path, compile=False)
gender_classifier = load_model(gender_model_path, compile=False)

Instructions for updating:
Colocations handled automatically by placer.


In [6]:
# getting input model shapes for inference
emotion_target_size = emotion_classifier.input_shape[1:3]
gender_target_size = gender_classifier.input_shape[1:3]

In [8]:
np.set_printoptions(precision=3)
np.set_printoptions(suppress=True)
def converter(x):

    #x has shape (batch, width, height, channels)
    return (0.21 * x[:,:,:1]) + (0.72 * x[:,:,1:2]) + (0.07 * x[:,:,-1:])

In [10]:
target_size=None
from skimage import io
from skimage.color import rgb2gray
from skimage import img_as_ubyte
import random


In [11]:
def gender_predict(img):
    gray_image=converter(img)
    rgb_image=np.squeeze(img)
    rgb_image = rgb_image.astype('uint8')
    
    gray_image = np.squeeze(gray_image)
    gray_image = gray_image.astype('uint8')
    faces = detect_faces(face_detection, gray_image)

    if len(faces)==0:
        rand_index=random.randint(0,1)
        ret_arr=np.zeros((1,2))
        ret_arr[0][rand_index]=1.0
        return ret_arr
    x1, x2, y1, y2 = apply_offsets(faces[0], gender_offsets)
    
    rgb_face = rgb_image[y1:y2, x1:x2]
    rgb_face = cv2.resize(rgb_face, (gender_target_size))

    rgb_face = preprocess_input(rgb_face, False)
    rgb_face = np.expand_dims(rgb_face, 0)
    
    return gender_classifier.predict(rgb_face)

In [12]:
def emotion_predict(img):
    gray_image=converter(img)
    # print(gray_image.shape)
    gray_image = np.squeeze(gray_image)
    gray_image = gray_image.astype('uint8')
    faces = detect_faces(face_detection, gray_image)

    if len(faces)==0:
        rand_index=random.randint(0,6)
        ret_arr=np.zeros((1,7))
        ret_arr[0][rand_index]=1.0
        return ret_arr
    
    x1, x2, y1, y2 = apply_offsets(faces[0], emotion_offsets)
    gray_face = gray_image[y1:y2, x1:x2]
    gray_face = cv2.resize(gray_face, (emotion_target_size))

    gray_face = preprocess_input(gray_face, True)
    gray_face = np.expand_dims(gray_face, 0)
    gray_face = np.expand_dims(gray_face, -1)
    return emotion_classifier.predict(gray_face)

In [16]:
data_path='../../Dataset/img_align_celeba/'
output_dict={}
for file in tqdm(os.listdir(data_path)):
    rgb_img = io.imread(os.path.join(data_path,file))
    try:
        output=gender_predict(rgb_img)[0]
    except:
        print(file)
    output_dict[file]={gender_labels[i]:np.round(output[i]*100,2) for i in range(len(gender_labels))}

100%|██████████| 1000/1000 [00:26<00:00, 37.06it/s]


In [17]:
import json

with open('gender_results_1000.json', 'w') as fp:
    json.dump(output_dict, fp)

In [18]:
data_path='../../Dataset/img_align_celeba/'
output_dict={}
for file in tqdm(os.listdir(data_path)):
    rgb_img = io.imread(os.path.join(data_path,file))
    output=emotion_predict(rgb_img)[0]
    output_dict[file]={emotion_labels[i]:np.round(output[i]*100,2) for i in range(len(emotion_labels))}

100%|██████████| 1000/1000 [00:47<00:00, 21.09it/s]


In [19]:
import json

with open('emotion_results_1000.json', 'w') as fp:
    json.dump(output_dict, fp)

### LIME Masks

In [20]:
import lime 
from lime import lime_image
from math import ceil,floor
import matplotlib.pyplot as plt
from skimage.segmentation import mark_boundaries
%matplotlib inline
explainer = lime_image.LimeImageExplainer()

In [21]:
def check_one(grid):
    total=sum(sum(np.ones(grid.shape)))
    sm=sum(sum(grid))
    return sm/total

def mask_transform(mask_arr,grid_size=24):
    grid_num=floor(mask_arr.shape[0]/grid_size)
    h=grid_size
    w=grid_size
    ret_arr=np.zeros((h,w))
    for i in range(h-1):
        for j in range(w-1):
            grid=mask_arr[grid_num*i:grid_num*(i+1),grid_num*j:grid_num*(j+1)]
            ret_arr[i][j]=int(check_one(grid) > 0.3)
    
    for i in range(h-1):
        grid=mask_arr[grid_num*i:grid_num*(i+1),grid_num*(w-1):]
        ret_arr[i][w-1]=int(check_one(grid) > 0.3)
    
    for j in range(w-1):
        grid=mask_arr[grid_num*(h-1):,grid_num*j:grid_num*(j+1)]
        ret_arr[h-1][j]=int(check_one(grid) > 0.3)
    return ret_arr

In [28]:
def generate_mask_gender(image):
    explanation = explainer.explain_instance(image, gender_predict, top_labels=2, hide_color=0, num_samples=100,batch_size=1)
    ret_list=[]
    for label in explanation.top_labels:
        temp, mask = explanation.get_image_and_mask(label, positive_only=True, num_features=12, hide_rest=False)
        ret_list+=[(mask_transform(mask),label)]
    return ret_list

def generate_mask_emotion(image):
    explanation = explainer.explain_instance(image, emotion_predict, top_labels=7, hide_color=0, num_samples=100,batch_size=1)
    ret_list=[]
    for label in explanation.top_labels:
        temp, mask = explanation.get_image_and_mask(label, positive_only=True, num_features=12, hide_rest=False)
        ret_list+=[(mask_transform(mask),label)]
    return ret_list

In [43]:
for image in tqdm(os.listdir('../../Dataset/img_align_celeba/')):
    rgb_img=io.imread(os.path.join('../../Dataset/img_align_celeba/',image))
    emotion_masks=generate_mask_emotion(rgb_img)
    gender_masks=generate_mask_gender(rgb_img)
    outfile='../../lime_masks/'+image.split('.')[0]+'.json'
    image_output={}
    for arr,label in emotion_masks:
        out_list=arr.tolist()
        image_output[emotion_labels[label]]=out_list
    
    for arr,label in gender_masks:
        out_list=arr.tolist()
        image_output[gender_labels[label]]=out_list
        
    json.dump(image_output, codecs.open(outfile, 'w', encoding='utf-8'), sort_keys=True, indent=4)

100%|██████████| 1000/1000 [44:08<00:00,  1.09s/it]   


(24, 24)